In [38]:
import backtrader as bt
import alpaca_backtrader_api as api
import alpaca_trade_api as tradeapi
import sys
sys.path.insert(0, '../')
import pandas as pd
import talib as ta
import numpy as np
from config import key_id, secret_key

In [39]:
#import cross signal generator
from cross_signal import *

In [40]:
import random
from datetime import datetime, timedelta


# or a function
def gen_datetime(min_year=1900, max_year=2019):
    # generate a datetime in format yyyy-mm-dd hh:mm:ss.000000
    start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=365 * years)
    return start + (end - start) * random.random()

In [41]:
#trading data is called in the form of JSON
#https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2019-01-01/2019-02-01?apiKey=


api = tradeapi.REST(key_id, secret_key)

In [42]:
#generate list of random days into list
day_list=[]
day_after_list=[]
for i in range(500):
    rand_day=gen_datetime(2009)
    day_list.append(rand_day.strftime("%Y-%m-%d"))
    day_after=rand_day + timedelta(days=14)
    day_after_list.append(day_after.strftime("%Y-%m-%d"))

In [43]:
day_list[0]

'2019-08-31'

In [44]:
#generate random list of symbols
sym_list=['JACK']
# ,'AAPL','TSLA','JACK'
#for i in range(10):
    #rand_sym=random.random(range=len(symbols.csv))
    #symlist.append(rand_sym)

In [45]:
#Generate random day data for
df_list=[]
for sym in sym_list: 
    data_day_list=[]
    for i in range(len(day_list)):
        data_sym = api.polygon.historic_agg_v2(sym, 1, 'minute', _from=day_list[i], to=day_after_list[i]).df
        data_day_list.append(data_sym)
    df_list.append(data_day_list)

In [12]:
cross_sign=crossed_below(df_list[0][0].high,60)
cross_sign[0]

False

# Testing

In [46]:
#feed strat to cerebro

class oscillator_mfi(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        
        #crossing signal
        self.adosc=bt.talib.ADOSC(self.data.high,self.data.low,self.data.close,self.data.volume, fastperiod=3, slowperiod=10) 
        #1 for down, -1 for up
        self.cross_sig=bt.indicators.CrossOver(self.adosc, 0)
        
        
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.cross_sig== -1 and \
            self.adx > 25 and \
            self.mfi < 15: \
            #self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.cross_sig== 1 and \
            self.mfi > 90:
                self.sell(size=20)


In [14]:
#feed strat to cerebro

class oscillator_rsi(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
    #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        
        self.rsi=bt.talib.RSI(self.data.close, timeperiod=14 )
        
        #crossing signal
        self.adosc=bt.talib.ADOSC(self.data.high,self.data.low,self.data.close,self.data.volume, fastperiod=3, slowperiod=10) 
        #1 for down, -1 for up
        self.cross_sig=bt.indicators.CrossOver(self.adosc, 0)
        
        
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        

        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.cross_sig== -1 and \
            self.rsi < 29: 
            #self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.cross_sig== 1 and \
            self.fastd > 70  and \
            self.rsi > 80:
                self.sell(size=20)


SyntaxError: unexpected character after line continuation character (<ipython-input-14-76d567556340>, line 56)

In [456]:
#feed strat to cerebro

class Scalpy3(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=10)
        self.emaclose=bt.indicators.EMA(self.data.close, period=10)
        self.emalow=bt.indicators.EMA(self.data.low, period=10)
        
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        
        
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=14, slowk_period=4, slowd_period=4)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.adx > 25 and \
            self.fastk < 20 and \
            self.fastd < 20 and \
            self.cci <= -150.0:
                self.buy(size=20)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.fastk > 95 and \
            self.fastd > 95  and \
            self.cci >= 150.0 :
                self.sell(size=20)


In [580]:
#feed strat to cerebro

class Scalpy4(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=10)
        self.emaclose=bt.indicators.EMA(self.data.close, period=10)
        self.emalow=bt.indicators.EMA(self.data.low, period=10)
        
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        self.ultosc=bt.talib.ULTOSC(self.data.high,self.data.low,self.data.close)
        
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=14, slowk_period=4, slowd_period=4)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        self.bband = bt.indicators.BBands(self.datas[0])
        self.bband_low=self.bband.lines.bot
        self.bband_high = self.bband.lines.top
        self.bband_mid= self.bband.lines.mid
        
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.bband_low and \
            self.adx > 25 and \
            self.fastk < 20 and \
            self.fastd < 20 and \
            self.cci <= -150.0:
                self.buy(size=20)
                
                
        else:
            if self.data.open > self.bband_high and \
            self.fastk > 95 and \
            self.fastd > 95  and \
            self.cci >= 150.0 :
                self.sell(size=20)


In [409]:
#feed strat to cerebro

class Scalp4(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=5)
        self.emaclose=bt.indicators.EMA(self.data.close, period=5)
        self.emalow=bt.indicators.EMA(self.data.low, period=5)
        self.rsi=bt.talib.RSI(self.data.close, timeperiod=14 )
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        
        
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=5, slowk_period=3, slowd_period=3)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #1 for down, -1 for up
        self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.adx > 30 and \
            self.rsi < 29  and \
            self.fastd < 30 and \
            self.fastk > self.fastd and \
            self.fastk < 30:
                self.buy(size=20)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.adx > 25 and \
            self.fastd > 70  and \
            self.rsi > 80  and \
            self.fastk > 70:
                self.sell(size=20)


In [513]:
#feed strat to cerebro

class Scalp5(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=5)
        self.emaclose=bt.indicators.EMA(self.data.close, period=5)
        self.emalow=bt.indicators.EMA(self.data.low, period=5)
        self.rsi=bt.talib.RSI(self.data.close, timeperiod=14 )
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        
        
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=5, slowk_period=3, slowd_period=3)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #1 for down, -1 for up
        self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.mfi < 25  and \
            self.adx > 30 and \
            self.rsi < 29  and \
            self.fastd < 30 and \
            self.fastk > self.fastd and \
            self.fastk < 30:
                self.buy(size=20)
                
        else:
            if self.mfi > 75 and \
            self.fastd > 70  and \
            self.rsi > 80  and \
            self.fastk > 70:
                self.sell(size=20)


In [376]:
"""self.cross_sig_below=crossed_below(self.adosc,0)
self.cross_sig_above=crossed_above(self.adosc,0)
        
        
    def next(self):
        if not self.position: #not in the market
            if self.cross_sig_above== True and \
            self.mfi < 30: \
            #self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.cross_sig_below== True and \
            self.mfi > 85:
                self.sell(size=20)"""

'self.cross_sig_below=crossed_below(self.adosc,0)\nself.cross_sig_above=crossed_above(self.adosc,0)\n        \n        \n    def next(self):\n        if not self.position: #not in the market\n            if self.cross_sig_above== True and             self.mfi < 30:             #self.minusdi > self.plusdi:\n                self.buy(size=20)\n                \n                \n        else:\n            if self.cross_sig_below== True and             self.mfi > 85:\n                self.sell(size=20)'

In [377]:
#feed strat to cerebro

class volumestrat7(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        self.ultosc=bt.talib.ULTOSC(self.data.high,self.data.low,self.data.close)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        
        #creat stochastic momentum indicators 
        #1 for down, -1 for up
        #self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.mfi < 20 and \
            self.ultosc < 20 and \
            self.minusdi > 25 and \
            self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.mfi > 80  and \
            self.ultosc > 80  and \
            self.minusdi > 25 and \
            self.minusdi < self.plusdi: 
                self.sell(size=20)


In [427]:
#feed strat to cerebro

class volumestrat8(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        self.ultosc=bt.talib.ULTOSC(self.data.high,self.data.low,self.data.close)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        
        #creat stochastic momentum indicators 
        #1 for down, -1 for up
        #self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.mfi < 23 and \
            self.adx > 25 and \
            self.ultosc < 23 and \
            self.minusdi > 25 and \
            self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.mfi > 82  and \
            self.ultosc > 80  and \
            self.minusdi < self.plusdi: 
                self.sell(size=20)


In [581]:
# Run over everything
SQN_dict_list=[] 
tradeanalysis_dict_list=[]
ret_dict_list=[]
trans_dict_list=[]
pnl_list=[]

for i in range(len(df_list[0])):
    #create backtrader
    cerebro = bt.Cerebro()

    #cerebro.addstrategy(SmaCross)
    #cerebro.addstrategy(Scalp)

    #Add slippage
    cerebro.broker = bt.brokers.BackBroker(slip_perc=0.005) 

    #set up cerebro

    cerebro.broker.setcash(100000)

    cerebro.broker.setcommission(commission=0.0)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=50)

    # Add the analyzers we are interested in
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="ret")
    cerebro.addanalyzer(bt.analyzers.PeriodStats, _name="pstat")
    cerebro.addanalyzer(bt.analyzers.Transactions, _name="trans")


    # Add the observers we are interested in 
    #this adds more things to the plot on top of the stocks chart
    cerebro.addobserver(bt.observers.Broker)
    cerebro.addobserver(bt.observers.BuySell)
    
    
    
    
    cerebro.addstrategy(Scalpy4)
    datacere = bt.feeds.PandasData(dataname=df_list[0][i])
    cerebro.adddata(datacere,name=day_list[i])
#    print(day_list[i]+' added')
    test=cerebro.run()
    
    
    
    
    
    SQN_dict=test[0].analyzers.sqn.get_analysis() 
    SQN_dict_list.append(SQN_dict)
    
    tradeanalysis_dict=test[0].analyzers.ta.get_analysis()
    tradeanalysis_dict_list.append(tradeanalysis_dict)
    
    ret_dict=test[0].analyzers.ret.get_analysis()
    ret_dict_list.append(ret_dict)
    
    trans_dict=test[0].analyzers.trans.get_analysis()
    trans_dict_list.append(trans_dict_list)
    

In [582]:
SQN_total=0
for i in range(len(SQN_dict_list)):    
    SQN_total=SQN_total+SQN_dict_list[i]['sqn']

SQN_avg=SQN_total/len(SQN_dict_list)

In [583]:
SQN_avg

0.29738150440321737

In [584]:
tradeanalysis_dict_list[0]

AutoOrderedDict([('total',
                  AutoOrderedDict([('total', 6), ('open', 1), ('closed', 5)])),
                 ('streak',
                  AutoOrderedDict([('won',
                                    AutoOrderedDict([('current', 0),
                                                     ('longest', 3)])),
                                   ('lost',
                                    AutoOrderedDict([('current', 1),
                                                     ('longest', 1)]))])),
                 ('pnl',
                  AutoOrderedDict([('gross',
                                    AutoOrderedDict([('total',
                                                      56.60000000000025),
                                                     ('average',
                                                      11.32000000000005)])),
                                   ('net',
                                    AutoOrderedDict([('total',
                                      

In [585]:
win_total=0
total_won_pnl=0
avg_won=0

lost_total=0
total_lost_pnl=0
avg_lost=0

total_pnl=0
total_avg=0

for i in range(len(df_list[0])):
    try:
        win_total=win_total+tradeanalysis_dict_list[i]['won']['total']
        total_won_pnl=total_won_pnl+tradeanalysis_dict_list[i]['won']['pnl']['total']
        avg_won=avg_won+tradeanalysis_dict_list[i]['won']['pnl']['average']

        lost_total=lost_total+tradeanalysis_dict_list[i]['lost']['total']
        total_lost_pnl=total_lost_pnl+tradeanalysis_dict_list[i]['lost']['pnl']['total']
        avg_lost=avg_lost+tradeanalysis_dict_list[i]['lost']['pnl']['average']

        total_pnl=total_pnl+tradeanalysis_dict_list[i]['pnl']['gross']['total']
        total_avg=total_avg+tradeanalysis_dict_list[i]['pnl']['gross']['average']
    except:
        pass


In [586]:
win_total , total_won_pnl , avg_won 

(2166, 18632.80799999998, 4564.912320024417)

In [587]:
lost_total , total_lost_pnl , avg_lost

(1389, -14306.824000000011, -5138.35652857143)

In [588]:
total_pnl , total_avg

(4325.983999999996, 514.3876690698181)

In [589]:
win_total/(win_total+lost_total)

0.6092827004219409

In [590]:
total_pnl/(total_pnl+lost_total)

0.7569547001356433

In [591]:
avg_won/(-avg_lost)

0.8883992955026725

In [592]:
# print the SQN analyzers  SystemQualityNumber
#test[0] is the first strategy used. For multiple strategies, use corresponding number in the list.
#SQN_dict=test[0].analyzers.sqn.get_analysis() 
#tradeanalysis_dict=test[0].analyzers.ta.get_analysis()
#ret_dict=test[0].analyzers.ret.get_analysis()
#trans_dict=test[0].analyzers.trans.get_analysis()


In [593]:
trans_dict

OrderedDict([(datetime.datetime(2010, 4, 12, 17, 19),
              [[20, 25.5, 0, '2010-04-12', -510.0]]),
             (datetime.datetime(2010, 4, 12, 18, 55),
              [[-20, 25.33, 0, '2010-04-12', 506.59999999999997]]),
             (datetime.datetime(2010, 4, 13, 13, 47),
              [[20, 25.38, 0, '2010-04-12', -507.59999999999997]]),
             (datetime.datetime(2010, 4, 14, 15, 18),
              [[-20, 25.46, 0, '2010-04-12', 509.20000000000005]]),
             (datetime.datetime(2010, 4, 14, 17, 44),
              [[20, 25.751, 0, '2010-04-12', -515.02]]),
             (datetime.datetime(2010, 4, 14, 18, 35),
              [[-20, 25.97, 0, '2010-04-12', 519.4]]),
             (datetime.datetime(2010, 4, 15, 19, 52),
              [[20, 26.24, 0, '2010-04-12', -524.8]]),
             (datetime.datetime(2010, 4, 16, 18, 18),
              [[-20, 24.95, 0, '2010-04-12', 499.0]]),
             (datetime.datetime(2010, 4, 19, 18, 16),
              [[20, 24.48, 0, '201

In [594]:
len(trans_dict)

19